# MFCC PCA 실험 

In [1]:
import pandas as pd 

train_merge_pca = pd.read_csv('./iotid20_train_pca+mfcc.csv')
test_merge_pca =  pd.read_csv('./iotid20_test_pca+mfcc.csv')

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.iloc[:, :-1].values  # 마지막 열을 제외한 데이터
        self.labels = dataframe.iloc[:, -1].values  # 마지막 열을 라벨 데이터로 사용

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {'data': torch.tensor(self.data[idx], dtype=torch.float32),
                  'label': torch.tensor(self.labels[idx], dtype=torch.long)}
        return sample

In [3]:
train_dataset_pca = CustomDataset(train_merge_pca)
train_loader_pca = DataLoader(train_dataset_pca, batch_size=64, shuffle=True, drop_last=True)

test_dataset_pca = CustomDataset(test_merge_pca)
test_loader_pca = DataLoader(test_dataset_pca, batch_size=64, shuffle=False, drop_last=True)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from resnet_1d import *

USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)

def create_models(num_classes=9):
    model_18 = resnet18(num_classes=9, pretrained=False)
    model_34 = resnet34(num_classes=9, pretrained=False)
    
    return model_18,model_34

model_18, model_34 = create_models()

다음 기기로 학습합니다: cuda


In [5]:
import os 
model = model_18

# resnet18
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_18.parameters(), lr=2e-4)

## train 및 valid
os.makedirs('./result', exist_ok=True)
from sklearn.metrics import f1_score, classification_report
import numpy as np
from tqdm.notebook import trange
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_loader_pca)
valid_loss = np.Inf  # train acc
f1_ = 0  # val f1
epoch_in = trange(100, desc='training')
best_acc=0



for epoch in epoch_in:
    model.to(device)
    model.train()
    running_loss = 0.0
    correct = 0
    total=0
    
    preds_ = []
    targets_ = []

    for batch_idx, train_dict in enumerate(train_loader_pca):

        inputs = train_dict['data'].to(device).float()
        inputs = inputs.reshape(64,1,1,12)
        labels = train_dict['label'].to(device).long()

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==labels).item()
        total += labels.size(0)
        # if (batch_idx) % 1000 == 0:
        #     print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        #            .format(epoch, n_epochs, batch_idx, total_step, loss.item()))

    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss):.4f}, train acc: {(100 * correct / total):.4f}')

    
    batch_loss = 0
    total_t=0
    correct_t=0    
    
    with torch.no_grad():
        model.to(device)
        model.eval()
        for test_dict in (test_loader_pca):
            data_t = test_dict['data'].to(device).float()
            data_t = data_t.reshape(64,1,1,12)
            #print(data_t)
            
            target_t =  test_dict['label'].to(device).long()
            #print(target_t)
            
            outputs_t = model(data_t)
            
            ####################### f1 score ################################
            pred = outputs_t.argmax(dim=1).to(device)
            target = target_t.view_as(pred).to(device)

            preds_.append(pred)
            targets_.append(target)
            
            # f1_score += f1(pred, target)
            ##################################################################
            
            
            
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)

        #print(total_t)

        val_acc.append(100 * correct_t / total_t)
        val_loss.append(batch_loss/len(test_loader_pca))
        network_learned = batch_loss < valid_loss_min

        preds_ = torch.cat(preds_).detach().cpu().numpy()
        targets_ = torch.cat(targets_).detach().cpu().numpy()

        f1score = f1_score(targets_,preds_,  average='macro')
        if best_acc < f1score:
            best_acc = f1score
            with open("./result/iot20d_pca+mfcc_res18_0123.txt", "a") as text_file:
                print('epoch=====',epoch, file=text_file)
                print(classification_report(targets_, preds_, digits=4), file=text_file)
            torch.save(model, f'./result/iot20d_pca+mfcc_res18_0123.pt') 
        epoch_in.set_postfix_str(f"epoch = {epoch},  f1_score = {f1score}, best_f1 = {best_acc}")

training:   0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\user\AppData\Local\Temp\ipykernel_18276\3543638301.py:15: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  'label': torch.tensor(self.labels[idx], dtype=torch.long)}



train loss: 0.1364, train acc: 94.6268


C:\Users\user\AppData\Local\Temp\ipykernel_18276\3543638301.py:15: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  'label': torch.tensor(self.labels[idx], dtype=torch.long)}



train loss: 0.0899, train acc: 98.4953


...


train loss: 0.0141, train acc: 99.8655


C:\Users\user\AppData\Local\Temp\ipykernel_18276\3543638301.py:15: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  'label': torch.tensor(self.labels[idx], dtype=torch.long)}



train loss: 0.0138, train acc: 99.9167


C:\Users\user\AppData\Local\Temp\ipykernel_18276\3543638301.py:15: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  'label': torch.tensor(self.labels[idx], dtype=torch.long)}


KeyboardInterrupt: 

# Mel spectrogram 이미지 생성 

# Test dataset

In [11]:
import os

# 이미지 저장 디렉토리 생성
output_directory_n = './melspec/test/normal'
os.makedirs(output_directory_n, exist_ok=True)

output_directory_1= './melspec/test/label1'
os.makedirs(output_directory_1, exist_ok=True)

output_directory_2= './melspec/test/label2'
os.makedirs(output_directory_2, exist_ok=True)

output_directory_3= './melspec/test/label3'
os.makedirs(output_directory_3, exist_ok=True)

output_directory_4= './melspec/test/label4'
os.makedirs(output_directory_4, exist_ok=True)

output_directory_5= './melspec/test/label5'
os.makedirs(output_directory_5, exist_ok=True)

output_directory_6= './melspec/test/label6'
os.makedirs(output_directory_6, exist_ok=True)

output_directory_7= './melspec/test/label7'
os.makedirs(output_directory_7, exist_ok=True)

output_directory_8= './melspec/test/label8'
os.makedirs(output_directory_8, exist_ok=True)

In [13]:
import os

# 이미지 저장 디렉토리 생성
output_directory_n = './melspec/train/normal'
#os.makedirs(output_directory_n, exist_ok=True)

output_directory_1= './melspec/train/label1'
#os.makedirs(output_directory_1, exist_ok=True)

output_directory_2= './melspec/train/label2'
#os.makedirs(output_directory_2, exist_ok=True)

output_directory_3= './melspec/train/label3'
#os.makedirs(output_directory_3, exist_ok=True)

output_directory_4= './melspec/train/label4'
#os.makedirs(output_directory_4, exist_ok=True)

output_directory_5= './melspec/train/label5'
#os.makedirs(output_directory_5, exist_ok=True)

output_directory_6= './melspec/train/label6'
#os.makedirs(output_directory_6, exist_ok=True)

output_directory_7= './melspec/train/label7'
#os.makedirs(output_directory_7, exist_ok=True)

output_directory_8= './melspec/train/label8'
#os.makedirs(output_directory_8, exist_ok=True)

In [14]:
import pandas as pd 

train_df = pd.read_csv('./iotid20_train_pca+mfcc.csv')

In [15]:
zero_label_rows = train_df[train_df['label'] == 0] # 정상데이터 
one_label_rows = train_df[train_df['label'] == 1] # 비정상 데이터 
two_label_rows = train_df[train_df['label'] == 2] # 비정상 데이터 
three_label_rows = train_df[train_df['label'] == 3] # 비정상 데이터 
four_label_rows = train_df[train_df['label'] == 4] # 비정상 데이터 
five_label_rows = train_df[train_df['label'] == 5] # 비정상 데이터 
six_label_rows = train_df[train_df['label'] == 6] # 비정상 데이터 
sev_label_rows = train_df[train_df['label'] == 7] # 비정상 데이터 
eig_label_rows = train_df[train_df['label'] == 8] # 비정상 데이터 

In [17]:
train__sample_val_normal = zero_label_rows.iloc[:,:-1].values
test_1 = one_label_rows.iloc[:,:-1].values
test_2 = two_label_rows.iloc[:,:-1].values
test_3 = three_label_rows.iloc[:,:-1].values
test_4 = four_label_rows.iloc[:,:-1].values
test_5 = five_label_rows.iloc[:,:-1].values
test_6 = six_label_rows.iloc[:,:-1].values
test_7 = sev_label_rows.iloc[:,:-1].values
test_8 = eig_label_rows.iloc[:,:-1].values

In [12]:
import librosa 
import librosa.display 
import matplotlib.pyplot as plt 
import tqdm 
import os
import numpy as np

SAMPLE_RATE = 1365 
DURATION = 0.75 
win_length = int(1365/40)
n_fft = 1024
hop_length = 4 
n_mels = 40

def run (df, path):
    for i in tqdm.tqdm(range(df.shape[0])):
        data = df[i:i+1, :].reshape(-1)
        S = librosa.feature.melspectrogram(y=data, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
        mel_spec_db = librosa.power_to_db(S, ref=np.max)

        # 플롯 생성
        plt.figure(figsize=(10, 4))  # 이미지 크기 조절
        librosa.display.specshow(mel_spec_db, x_axis='time', y_axis='mel',\
            sr=SAMPLE_RATE, hop_length=hop_length, cmap='viridis')

        # 눈금 및 레이블 제거
        plt.axis('off')

        # 이미지 저장
        plt.savefig(f'{path}/image_{i}.png', bbox_inches='tight', pad_inches=0)

        # 플롯 닫기 (메모리 누수 방지)
        plt.close()

run(train__sample_val_normal, output_directory_n)
run(test_1, output_directory_1)
run(test_2, output_directory_2)
run(test_3, output_directory_3)
run(test_4, output_directory_4)
run(test_5, output_directory_5)
run(test_6, output_directory_6)
run(test_7, output_directory_7)
run(test_8, output_directory_8)

  0%|          | 0/1000 [00:00<?, ?it/s]c:\Users\user\anaconda3\envs\mintorch1221\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=12
  warnings.warn(
100%|██████████| 1000/1000 [01:34<00:00, 10.59it/s]
